In [26]:
# global constants
g_maxAngVel = pi / 2 # move in azimuth or elevation at max rate of 90 deg / sec
g_maxScanRate = 1000 # scans points at the rate of 1000 pts/sec

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;


# function to simulate a scan from one spherical coord to another
# input: from: spherical coordinate triple (radius, theta, phi) to start from
#        to:   spherical coordinate triple (radius, theta, phi) to end at
# output: n x 3 list of spherical points
function scan_from_to(from, to)
    delta = to .- from
    time = maximum(abs.(delta[2:3])) / g_maxAngVel
    qty = floor(time * g_maxScanRate)
    rgs = linspace.(from, to, qty)
    return [collect(rgs[1]) collect(rgs[2]) collect(rgs[3])]
end

# function to simulate a scan through a list of (spherical) points
# input:  list of points (spherical) in the order to be visited
# output: list of points (spherical) of the simulated scan points
function scan_tour(pts)
    ret = []
    for i = 1:size(pts,1)-1
        line = scan_from_to(pts[i,:], pts[i+1,:])
        ret = [ret; line]
    end
    return ret
end

function test_transforms()
    # load the feature file
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # convert to spherical
    pts_mcs = pcs_to_mcs_xyz(lr_tfm, pts_pcs) # convert feature locations to MCS
    pts_lr = xyz_to_sph(pts_mcs) # convert feature locations to spherical

    # round trip as a test
    pts_mcs_back = sph_to_xyz(pts_lr)
    pts_pcs_back = mcs_to_pcs_xyz(lr_tfm, pts_mcs_back)

    # points should be the same
    diff = pts_pcs - pts_pcs_back
    display(maximum(diff))
end

function test_scan()
    # load data.
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # shuffle an index vector
    idx = Vector(1:size(pts_pcs,1))
    rng = MersenneTwister(1234)
    idx = shuffle(rng, idx)
    
    # shuffle input pts and convert them to spherical
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs[idx,:]))

    # generate scan data
    scan = scan_tour(sph)
    
    # back to pcs
    scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
    println(size(scan))
    
    # save
    writecsv("scan1.csv", scan)
end

test_transforms()
test_scan()

8.881784197001252e-16

(12146, 3)


In [48]:
using JuMP, Gurobi

# function that computes the travel time from spherical coordinate 'from' to 'to'
# (assumes instantaneous acceleration to max velocity)
travel_time(from, to) = maximum(abs.((to .- from)[2:3])) / g_maxAngVel

# function that takes a list of nx3 spherical coordinates and returns an n x n 
# matrix where A(i,j) where  the time it takes to move from point i to point j
cost_matrix(sph) = [travel_time(sph[i,:],sph[j,:]) for i=1:size(sph,1), j=1:size(sph,1)]

#function find_subtour(idxs, at)
#    display(@sprintf("find_subtour(%d)\n", at))
#    cur_subtour = [idxs[at]]
#    while true
#        at = idxs[at] # move to next
#        display(cur_subtour)
#        if idxs[at] == cur_subtour[1]
#            display(@sprintf("returning subtour\n"))
#            display(cur_subtour)
#            return cur_subtour
#        end
#        push!(cur_subtour, idxs[at])
#        display(cur_subtour)
#    end
#end    
#
#function find_subtours(idxs)
#    ret = []
#    cur_subtour = []
#    remaining = idxs
#    while length(remaining) > 0
#        at = remaining[1]
#        push!(ret, find_subtour(idxs, at))
#        remaining = setdiff(remaining, subtour)
#    end
#    return ret
#end

function find_subtour(idxs,start)
    #display(@sprintf("find_subtour(%d)\n", start))
    subtour = [start]
    cnt = 0
    while true
        next = idxs[subtour[end]]
        push!(subtour,next)
        if start == next
            break
        end
        cnt += 1
        if (cnt > 100)
            display("Error1")
            break;
        end
    end
    #display(@sprintf("returing subtour\n"))
    #display(subtour)        
    return subtour
end

function find_subtours(idxs)
    remaining = [i for i=1:length(idxs)]
    subtours = []
    cnt = 0
    while length(remaining) > 0
        subtour = find_subtour(idxs, remaining[1])
        push!(subtours, subtour)
        remaining = setdiff(remaining, subtour)
        cnt += 1
        if (cnt > 100)
            display("Error")
            break
        end
    end
    return subtours
end

function min_path(features, lr_location)
    # xyz location of features
    pts_pcs = convert(Array{Float64, 2}, features[:,2:4]) # feature locations in PCS

    # spherical coordinates with origin at laser radar
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_location, pts_pcs))
    
    n = size(sph,1) # number of features
    cost = cost_matrix(sph) # cost to travel from feature i to feature j

    println(n)
    println(size(cost))
    # TSP solve
    m = Model(solver=GurobiSolver(OutputFlag=0))
    @variable(m, z[1:n, 1:n], Bin)
    @objective(m, Min, sum(z.*cost)) # minimize the sum of movement times
    @constraint(m, [i in 1:n], sum(z[i,:]) == 1) # leave each feature once
    @constraint(m, [j in 1:n], sum(z[:,j]) == 1) # arrive at each feature once
    @constraint(m, [ii in 1:n], z[ii,ii] == 0) # can't transition to self
    
    # explicitly forbid 2-loops right away.
    #@constraint(m, [i=1:n,j=1:n], z[i,j] + z[j,i] <= 1)
    
    # MTZ variables and constraints
    @variable(m, u[1:n])
    @constraint(m, [i=1:n, j=2:n], u[i] - u[j] + n*z[i,j] <= n-1 )
    
    
    # if using MTZ constraints (above), then this will only do 1 iteration
    # comment out MTZ constraints above to use lazy subtour elimination
    iter = 0
    soln = []
    solved = false
    while iter < 100 && !solved
        iter += 1
        
        #solve
        t1 = @timed(solve(m))
        soln = getvalue(z)

        # check for subtours
        idxs = [find(soln[i,:] .>= 0.9)[1] for i=1:n]
        subtours = find_subtours(idxs)
        display(@sprintf("Iteration %d contained %d subtours and solved in %f seconds\n", iter, length(subtours), t1[2]))
        if length(subtours) == 1
            solved = true;
        else
            # add constraints to prohibit these subtours
            for subtour in subtours
                len = length(subtour)
                # prohibit these particular features from being a tour
                @constraint(m, sum( z[subtour[i],subtour[i+1]] for i = 1:len-1 ) <= len-2)
                # and prohibit the reverse
                @constraint(m, sum( z[subtour[i+1],subtour[i]] for i = 1:len-1 ) <= len-2)
            end            
        end        
    end

    if solved
        display(@sprintf("solved in %d iterations\n", iter))
        # extract feature order
        order = [1 for i=1:n]
        for i in 2:n
            order[i] = find(soln[order[i-1],:] .== 1)[1]
        end
        display(order)

        # simulate a scan of the features
        # scan = scan_tour(sph[order,:])
        ret = []
        for i = 1:n
            to = find(soln[i,:] .== 1)[1]
            @printf("scan from %d to %d\n", i, to)
            line = scan_from_to(sph[i,:], sph[to,:])
            ret = [ret; line]
        end
        # back to pcs
        return mcs_to_pcs_xyz(lr_location, sph_to_xyz(ret))
    else
        display(@sprintf("not solved in %d iterations\n", iter))
    end
            
    
end


features = readcsv("50_Holes_FrontLeft.csv");
lr_position = readcsv("Location1.csv") # transformation matrix for LR's positionend
scan = @time(min_path(features[1:20,:], lr_position))
display(size(scan))
writecsv("scan2.csv", scan)


"Iteration 1 contained 1 subtours and solved in 14.785225 seconds\n"

"solved in 1 iterations\n"

20-element Array{Int64,1}:
  1
  5
  6
 11
 17
 14
 15
 16
 20
 19
 18
  2
 12
 13
  3
 10
  9
  8
  7
  4

20
(20, 20)


(824, 3)

Academic license - for non-commercial use only
scan from 1 to 5
scan from 2 to 12
scan from 3 to 10
scan from 4 to 1
scan from 5 to 6
scan from 6 to 11
scan from 7 to 4
scan from 8 to 7
scan from 9 to 8
scan from 10 to 9
scan from 11 to 17
scan from 12 to 13
scan from 13 to 3
scan from 14 to 15
scan from 15 to 16
scan from 16 to 20
scan from 17 to 14
scan from 18 to 2
scan from 19 to 18
scan from 20 to 19
 15.457331 seconds (323.17 k allocations: 15.486 MiB, 0.07% gc time)
